In [6]:
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression
from sklearn.model_selection import GridSearchCV
from joblib import dump, load
import scipy.io
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [3]:
def Pythoninit():
    from sklearn.neural_network import MLPRegressor
    from sklearn.svm import SVR
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.linear_model import BayesianRidge, LinearRegression
    from sklearn.model_selection import GridSearchCV
    from joblib import dump, load
    import scipy.io
    import numpy as np
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_squared_error
    

In [4]:
Pythoninit()

## Auto adjusting Subrogated model 

In [11]:
def AutoSB( inputs_train,outputs_train):
    # firstmodel
    model_SVM=SVR()
    param_grid_SVM = { 'kernel':['linear','rbf'],
                  'C':[0.01,0.1,1,10,100],
                  'gamma':[1,10,100,1000,10000,'auto']}
    gridSVM = GridSearchCV(model_SVM,param_grid_SVM,refit=True,verbose=0,scoring='neg_mean_squared_error')
    gridSVM.fit(inputs_train,outputs_train)
    print(gridSVM.best_params_)
    print(gridSVM.best_score_)
    
    model_RFR=RandomForestRegressor()
    param_grid_RFR = { 'n_estimators':[200, 600, 1000, 1400, 1800],
                       'max_features':['auto', 'log2'],
                       'max_depth': [10, 50, 90, None],
                       'min_samples_leaf': [1, 2, 4],
                       'min_samples_split': [2, 5, 10]}
    
    gridRFR = GridSearchCV(model_RFR,param_grid_RFR,refit=True,verbose=0,scoring='neg_mean_squared_error')
    gridRFR.fit(inputs_train,outputs_train)
    print(gridRFR.best_params_)
    print(gridRFR.best_score_)
          
    model_BR= BayesianRidge(n_iter=500)
    param_grid_BR = {'alpha_1':[1e-4,1e-6,1e-7],
                     'alpha_2':[1e-4,1e-6,1e-7],
                     'lambda_1':[1e-4,1e-6,1e-7],
                     'lambda_2':[1e-4,1e-6,1e-7]}
    gridBR= GridSearchCV(model_BR,param_grid_BR,refit=True,verbose=0,scoring='neg_mean_squared_error')
    gridBR.fit(inputs_train,outputs_train)
    print(gridBR.best_params_)
    print(gridBR.best_score_)     

          
    return  gridSVM,gridRFR,gridBR
    
    
    

In [7]:
mat = scipy.io.loadmat('oneShotNonlinear.mat') 
mat['Expout'][1]
mat['Experimments'][1]

array([0.5 , 0.05, 0.5 , 0.4 , 0.5 , 0.5 ])

In [8]:
len(mat['Experimments'])

730

In [ ]:
mat = scipy.io.loadmat('nonlinealOscillatortrain.mat') 
mat['Expout'][1]
mat['Experiments'][1]

In [12]:
modeloSVM,ModeloRFR,ModeloBR=AutoSB( mat['Experimments'],mat['Expout'].reshape(730))

{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
-0.00731827646932348


KeyboardInterrupt: 

In [7]:
modeloSVM.best_score_

-0.07055820741242444

In [8]:
ModeloRFR.best_score_

-0.8213542928865815

In [9]:
ModeloBR.best_score_

-5.0690954907114865

In [10]:
dump(modeloSVM,'modeloSVM.joblib')
dump(ModeloRFR,'ModeloRFR.joblib')
dump(ModeloBR,'ModeloBR.joblib')

['ModeloBR.joblib']

In [9]:
modeloSVM.predict(mat['Experiments'])

array([  157.30325767, 23832.48527411,   157.30325767, 23832.48527408,
        1313.26744524, 24988.44946173,  1313.26744524, 24988.44946173,
         157.32815169, 23832.51016818,   157.32815169, 23832.51016817,
        1313.29233926, 24988.47435573,  1313.29233926, 24988.47435571,
         157.76450252, 23832.94651901,   157.76450252, 23832.94651898,
        1313.72869009, 24988.91070658,  1313.72869009, 24988.91070658,
         157.78939654, 23832.97141304,   157.78939655, 23832.97141302,
        1313.75358411, 24988.93560056,  1313.75358411, 24988.93560061])

In [11]:
ModeloRFR.predict(mat['Experiments'])

array([1.61339031e+02, 4.70320654e+03, 5.85241446e+02, 4.55878429e+03,
       7.10061554e+03, 7.83283899e+04, 9.33732931e+03, 8.50929380e+04,
       7.15972904e+02, 8.86097664e+03, 5.90442193e+02, 8.89046934e+03,
       4.34173882e+03, 3.78466831e+04, 2.89707073e+03, 3.48778396e+04,
       1.00898271e+03, 1.26925951e+04, 2.44006853e+03, 1.58875695e+04,
       1.98019724e+04, 2.59582486e+05, 3.66021130e+04, 2.57524421e+05,
       2.52447055e+03, 2.81519858e+04, 2.30490940e+03, 2.67801853e+04,
       1.35061097e+04, 1.14442305e+05, 8.09527866e+03, 1.06117727e+05])

In [14]:
r2_score(ModeloRFR.predict(mat['Experiments']),mat['Expout'].reshape(32))

0.9470523135672089

In [23]:
def modelEvaluation(moldelList,Experiments,Expout):
    LScore=[];
    MSElist = []
    for model in moldelList:
        MSE = mean_squared_error(Expout, model.predict(Experiments), multioutput='raw_values')
        Score = r2_score(model.predict(Experiments),Expout);
        LScore.append(Score);
        MSElist.append(MSE);
    return LScore,MSElist

In [34]:
modelEvaluation([modeloSVM,ModeloRFR,ModeloBR],mat['Experiments'],mat['Expout'].reshape(32))

([-38.95006813000528, 0.926904087913322, -3.8304514274910337],
 [array([5.61149141e+09]), array([2.9048216e+08]), array([4.58237804e+09])])

In [19]:
exp= scipy.io.loadmat('PruebaMC.mat') 
exp

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on:',
 '__version__': '1.0',
 '__globals__': [],
 'Test': array([[1.19517539e+04, 3.61000000e-01, 5.55000000e+01, 6.65000000e+01,
         1.10000000e+01, 7.50000000e+00, 4.64486273e-01],
        [1.56970577e+04, 3.20000000e-01, 5.55000000e+01, 6.65000000e+01,
         1.10000000e+01, 7.00000000e+00, 4.83116601e-01],
        [1.02417509e+04, 2.87000000e-01, 6.38000000e+01, 7.60000000e+01,
         1.28000000e+01, 9.50000000e+00, 7.92548842e-01],
        [8.07256097e+02, 2.26000000e-01, 4.00000000e+01, 4.25000000e+01,
         8.00000000e+00, 1.70000000e+01, 2.41201613e-01],
        [1.91790512e+03, 2.87000000e-01, 5.55000000e+01, 6.65000000e+01,
         1.10000000e+01, 1.65000000e+01, 6.23322989e-01],
        [1.47417675e+04, 2.26000000e-01, 6.38000000e+01, 7.60000000e+01,
         1.28000000e+01, 1.50000000e+01, 3.02642093e-01],
        [6.91434367e+02, 2.87000000e-01, 5.55000000e+01, 6.65000000e+01,
         1.10000000

In [35]:
modelEvaluation([modeloSVM,ModeloRFR,ModeloBR],exp['Test'],exp['Testout'].reshape(7))

([0.3247393264196463, -0.5264471081611763, -1.0672402318887668],
 [array([53953250.82121416]), array([8.4241066e+08]), array([1.15760078e+09])])

In [7]:
mat['Expout'].shape

(32, 1)

In [8]:
len(mat['Expout'])

32

In [13]:
def AutoSB( inputs_train,outputs_train):
    # firstmodel
    model_SVM=SVR()
    param_grid_SVM = { 'kernel':['linear','rbf'],
                  'C':[0.01,0.1,1,10,100],
                  'gamma':[1,10,100,1000,10000,'auto']}
    gridSVM = GridSearchCV(model_SVM,param_grid_SVM,refit=True,verbose=0,scoring='neg_mean_squared_error')
    gridSVM.fit(inputs_train,outputs_train)
    print(gridSVM.best_params_)
    print(gridSVM.best_score_)
    
    #model_RFR=RandomForestRegressor()
    #param_grid_RFR = { 'n_estimators':[200, 600, 1000, 1400, 1800],
    #                   'max_features':['auto', 'log2'],
    #                   'max_depth': [10, 50, 90, None],
    #                   'min_samples_leaf': [1, 2, 4],
    #                   'min_samples_split': [2, 5, 10]}
    
    #gridRFR = GridSearchCV(model_RFR,param_grid_RFR,refit=True,verbose=0,scoring='neg_mean_squared_error')
    #gridRFR.fit(inputs_train,outputs_train)
    #print(gridRFR.best_params_)
    #print(gridRFR.best_score_)
          
    #model_BR= BayesianRidge(n_iter=500)
    #param_grid_BR = {'alpha_1':[1e-4,1e-6,1e-7],
    #                 'alpha_2':[1e-4,1e-6,1e-7],
    #                 'lambda_1':[1e-4,1e-6,1e-7],
    #                'lambda_2':[1e-4,1e-6,1e-7]}
    #gridBR= GridSearchCV(model_BR,param_grid_BR,refit=True,verbose=0,scoring='neg_mean_squared_error')
    #gridBR.fit(inputs_train,outputs_train)
    #print(gridBR.best_params_)
    #print(gridBR.best_score_)     

          
    return  gridSVM
    
    
    

In [14]:
mat = scipy.io.loadmat('oneShotNonlinear.mat') 
mat['Expout'][1]
mat['Experimments'][1]

array([0.5 , 0.05, 0.5 , 0.4 , 0.5 , 0.5 ])

In [15]:
modeloSVM=AutoSB( mat['Experimments'],mat['Expout'].reshape(730))

{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}
-0.00731827646932348


In [33]:
Test = scipy.io.loadmat('nonlinealOscillatortest.mat') 
Test['Testout'][1]
Test['Test'][1]

array([1.1540791 , 0.11892145, 1.24815159, 0.49010832, 0.58382138,
       0.72897697])

In [34]:
Test

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on:',
 '__version__': '1.0',
 '__globals__': [],
 'Test': array([[1.29428454, 0.0811215 , 1.02853314, 0.43312975, 1.10198194,
         0.76297128],
        [1.1540791 , 0.11892145, 1.24815159, 0.49010832, 0.58382138,
         0.72897697],
        [1.41333736, 0.0652378 , 1.32581698, 0.50766849, 1.49613472,
         0.57817553],
        [0.94267827, 0.06066528, 1.46189808, 0.40092684, 1.27491046,
         1.31730322],
        [1.36869471, 0.05844358, 0.89978265, 0.45197408, 1.30006848,
         0.93141383],
        [1.41064759, 0.0681847 , 0.76380292, 0.4291078 , 0.63606856,
         1.36929221],
        [1.07970459, 0.10498602, 0.6449548 , 0.57060622, 1.12205513,
         0.85095238],
        [1.01324954, 0.0901808 , 0.57596669, 0.44798323, 0.62331893,
         0.68390779],
        [0.73995253, 0.09172671, 0.54965443, 0.58054322, 1.44478719,
         0.99086409],
        [0.98925264, 0.08377194, 1.40005385, 0.47384936, 0.

In [35]:
Test['Testout'].shape

(16, 1)

In [39]:
modelEvaluation([modeloSVM],Test['Test'],Test['Testout'].reshape(16))

([0.9872621594968167], [array([0.00320203])])